Subject's behavioural data during the training sessions and fMRI is preprocessed. The output of this pipeline is analysis dataframes

In [1]:
import os
import sys
from pathlib import Path
import glob
import getpass
import importlib

import numpy as np
import pandas as pd
import scipy.io as sio

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.cm import ScalarMappable
import matplotlib.cm as cm
import matplotlib.colors as mcolors

import utils
importlib.reload(utils)

# SET PATHS:
baseDir = os.path.join('/Users', getpass.getuser(), 'Desktop', 'Projects', 'bimanual_wrist')
fMRI_behavDir = os.path.join(baseDir,'data','fMRI','behavioural')
train_behavDir = os.path.join(baseDir,'data','training')
anaDir = os.path.join(baseDir,'analysis')
figDir = os.path.join(baseDir,'figures')

path = {'baseDir': baseDir,
        'fMRI_behavDir': fMRI_behavDir,
        'train_behavDir': train_behavDir,
        'anaDir': anaDir}



## Subject routine and prep

In [3]:
sn = 4

# load the behavioural dta of subject:
train = pd.read_table(os.path.join(train_behavDir, f's{sn:02d}', f'BimanualWrist_MR_{sn}.dat'))
fMRI = pd.read_table(os.path.join(fMRI_behavDir, f's{sn:02d}', f'BimanualWrist_MR_{sn}.dat'))

utils.dataset.subject_routine(sn, path, smooth_win_sz=0, fs=200)


Processing block 1
Processing trial 1
Processing trial 2


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
sn = 4
D = pd.read_csv(os.path.join(anaDir, f'bmw_train_{sn}.csv'))
D = D[D.GoodMovement == 1]
D_mov = pd.read_csv(os.path.join(anaDir, f'bmw_train_{sn}_mov.csv'))

# plot unimanual movements:
conds = D.cond_name.unique()
left_conds = [cond for cond in conds if 'left_' in cond]
right_conds = [cond for cond in conds if 'right_' in cond]

for cond in conds:
    BNs = D.BN[D.cond_name==cond].values.flatten()
    TNs = D.TN[D.cond_name==cond].values.flatten()
    
    # plot kinematic traces for left and right hands:
    fix, ax = plt.subplots(figsize=(10, 5))
    utils.please.draw_board(ax=ax)
    for i in range(len(BNs)):
        BN = BNs[i]
        TN = TNs[i]

        # kinematic traces:
        trial_mov = D_mov[(D_mov.BN==BN) & (D_mov.TN==TN)]
        radius_l = trial_mov['mov_5'].values.flatten()
        radius_r = trial_mov['mov_6'].values.flatten()
        angle_l = trial_mov['mov_7'].values.flatten()
        angle_r = trial_mov['mov_8'].values.flatten()

        radius = 5
        x_offset = radius+2
        y_offset = 0
        x_l = radius_l * np.cos(np.deg2rad(angle_l)) - x_offset
        y_l = radius_l * np.sin(np.deg2rad(angle_l)) + y_offset
        x_r = radius_r * np.cos(np.deg2rad(angle_r)) + x_offset
        y_r = radius_r * np.sin(np.deg2rad(angle_r)) + y_offset

        plt.plot(x_l, y_l, color='red', alpha=0.5)
        plt.plot(x_r, y_r, color='blue', alpha=0.5)
    plt.title(f'{cond}, subject {sn}')
    plt.xlabel('x')
    plt.ylabel('y')
    ax.set_xlim(-14, 14)
    ax.set_ylim(-7, 7)
    plt.savefig(os.path.join(figDir, 'behaviour', f'training_{sn}_{cond}.pdf'), bbox_inches="tight")
    plt.close()
